In [1]:
import mitsuba as mi
import drjit as dr 
import matplotlib.pyplot as plt
mi.set_variant("cuda_ad_rgb")

# Surface Reconstruction based on Photometric Stereo

## Get Image $I$

In [2]:
## get image I ##
bmp = mi.Bitmap('../scenes/img/normal-10-10.png')
I_t = mi.TensorXf(bmp)/256

## Set LED Configuration

In [3]:
import utils.led_ring

N_LED = 24
LED0_pos = mi.Point3f(34.41,0,-12.5)
LED_ring = utils.led_ring.LED_params(N_LED, LED0_pos)
P_LED = LED_ring.P
D_LED = LED_ring.D

## get LED intensity ##
A = 0.4
B = 0.5
# LED_ring.set_sine_intensity(A,B)
LED_ring.set_sine_intensity(A,B)


## Set Camera Configuration

In [4]:
Intrinsics = mi.Matrix3f([[422.3949, 0,313.5413],[0,422.4885,233.1770],[0,0,1]])
Z_camera = -44
image_res = (640,480)
cam_conner1 = dr.inverse(Intrinsics)@mi.Point3f(0,0,1)*-Z_camera
cam_conner2 = dr.inverse(Intrinsics)@mi.Point3f(image_res[0]-1,image_res[1]-1,1)*-Z_camera

# ### camera origin          ###
cam_origin = mi.Point3f(0,0,Z_camera)
cam_range = ((cam_conner1[0,0],cam_conner1[1,0]),(cam_conner2[0,0],cam_conner2[1,0]))
cam_fov= (cam_conner2[0,0]-cam_conner1[0,0],cam_conner2[1,0]-cam_conner1[1,0])
from utils import camera 
cam = camera.Camera(cam_origin, image_res,cam_range)

## Surface Reconstruction Iteration

In [ ]:
import numpy as np
from utils import iristac
from utils.depth_from_grad import frankotchellappa, fast_poisson
from decimal import Decimal

sensor = iristac.sensor(cam,LED_ring)

Z = None
N_Pixel = image_res[0]*image_res[1]

### Ground truth ###
scene_ref = mi.load_file('../scenes/iristac_ply.xml')
si_ref = cam.shot(scene_ref)
Z_ref = si_ref.p[2]


for i in range(6):
    if i>0:
        sensor.set_Z0(Z_t)
    normal = sensor.get_surface_normal(I_t)
    grad_x = np.reshape(-normal[0]/normal[2],(image_res[1],image_res[0]))
    grad_y = np.reshape(-normal[1]/normal[2],(image_res[1],image_res[0]))
    Z = fast_poisson(grad_x,grad_y)
    Z = Z*np.sqrt(cam_fov[0]*cam_fov[1]/N_Pixel)
    error_rms = np.mean(np.sqrt(((np.ravel(Z)-Z_ref))**2))
    error_std = np.std((np.ravel(Z)-Z_ref)**2)
    print("fast poisson solver reconstruction error:\n\
    rms error:   \t {0:.2E}\n\
    std  error:  \t {1:.2E}\n\
    ".format(Decimal(error_rms),Decimal(error_std))
    )
    Z_t = mi.TensorXf(np.ravel(Z),(cam.image_res[1],cam.image_res[0]))
